In [18]:
import os
import re
import gc
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import warnings

warnings.simplefilter('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', 200)

In [46]:
train = pd.read_csv('G:/kaggle竞赛GM之路！！！/竞赛ing/2021CCF系统认证风险预测/train_dataset.csv', sep='\t')
test = pd.read_csv('G:/kaggle竞赛GM之路！！！/竞赛ing/2021CCF系统认证风险预测/test_dataset.csv', sep='\t')

In [47]:
data = pd.concat([train, test])

#剔除常量
data.drop(['client_type', 'browser_source'], axis=1, inplace=True)

#填充缺失值
data['auth_type'].fillna('na', inplace=True)


#拆分地址
data['loc_1'] = data['location'].astype(str).apply(lambda x: json.loads(x)['first_lvl'])
data['loc_2'] = data['location'].astype(str).apply(lambda x: json.loads(x)['sec_lvl'])
data['loc_3'] = data['location'].astype(str).apply(lambda x: json.loads(x)['third_lvl'])

# 合并 action和auth_type
data['yanzh'] = data['action']+data['auth_type']

# 合并 ip, ip_location_type_keyword, ip_risk_level
data['ip_cat'] = data['ip'] + data['ip_location_type_keyword'] + data['ip_risk_level']

# 合并 bus_system_code, op_target
data['op'] = data['bus_system_code'] + data['op_target']

data.drop(['action','auth_type','ip','ip_location_type_keyword','ip_risk_level','bus_system_code','op_target'], axis=1, inplace=True)

#类别特征处理
for col in tqdm(['user_name', 'yanzh', 'ip_cat', 'location', 'device_model',
				'os_type', 'os_version', 'browser_type', 'browser_version',
				'op', 'loc_1', 'loc_2','loc_3']):
	lbl = LabelEncoder()
	data[col] = lbl.fit_transform(data[col])


# 核心特征挖掘
# 异常时间，短时高频，地址异常，设备异常

data['op_date'] = pd.to_datetime(data['op_date'])

# 时间差
data['op_ts'] = data['op_date'].values.astype(np.int64)
data = data.sort_values(by=['user_name', 'op_ts']).reset_index(drop=True)
data['last_ts'] = data.groupby(['user_name'])['op_ts'].shift(1)
data['ts_diff1'] = data['op_ts'] - data['last_ts']


# 是否超过23点
data['hour'] = data['op_date'].dt.hour
data['past_23'] = list(map(lambda x: 1 if x>=23 else 0,data['hour']))


# 同用户同类型的应用系统操作，验证方式、IP、地址、使用设备、操作系统、浏览器，
data = data.sort_values(by=['user_name', 'op_ts']).reset_index(drop=True)

for col in ['ip_cat', 'location', 'device_model', 'os_version', 'browser_version']:
	data['op_mean_' + str(col)] = data.groupby(['user_name','op'])[col].transform('mean')
	data['op_bias'] = data[col] - data['op_mean_' + str(col)]
	data.drop(['op_mean_' + str(col)],axis=1,inplace=True)


# 用户、验证方式、IP、地址、使用设备、操作系统、浏览器，分组时间差的统计量
for method in ['mean', 'max', 'min', 'std', 'sum', 'median','prod']:
	for col in ['user_name', 'ip_cat', 'location', 'device_model', 'os_version', 'browser_version']:
		data[f'ts_diff1_{method}_' + str(col)] = data.groupby(col)['ts_diff1'].transform(method)


data.drop(['location','op_ts','last_ts','hour'], axis=1, inplace=True)

  0%|          | 0/13 [00:00<?, ?it/s]

In [48]:
train = data[data['risk_label'].notna()]
test = data[data['risk_label'].isna()]

ycol = 'risk_label'
feature_names = list(filter(lambda x: x not in [ycol, 'session_id', 'op_date'], train.columns))

model = lgb.LGBMClassifier(objective='binary',
							boosting_type='gbdt',
							tree_learner='serial',
							num_leaves=2 ** 8,
							max_depth=16,
							learning_rate=0.2,
							n_estimators=10000,
							subsample=0.75,
							feature_fraction=0.55,
							reg_alpha=0.2,
							reg_lambda=0.2,
							random_state=0,
							is_unbalance=True,
							# scale_pos_weight=130,
							metric='auc')

oof = []
prediction = test[['session_id']]
prediction[ycol] = 0
df_importance_list = []

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(train[feature_names], train[ycol])):
	X_train = train.iloc[trn_idx][feature_names]
	Y_train = train.iloc[trn_idx][ycol]
	
	X_val = train.iloc[val_idx][feature_names]
	Y_val = train.iloc[val_idx][ycol]
	
	print('\nFold_{} Training ================================\n'.format(fold_id + 1))
	
	lgb_model = model.fit(X_train,
						Y_train,
						eval_names=['train', 'valid'],
						eval_set=[(X_train, Y_train), (X_val, Y_val)],
						verbose=500,
						eval_metric='auc',
						early_stopping_rounds=50)
	
	pred_val = lgb_model.predict_proba(X_val, num_iteration=lgb_model.best_iteration_)
	
	df_oof = train.iloc[val_idx][['session_id', ycol]].copy()
	df_oof['pred'] = pred_val[:, 1]
	oof.append(df_oof)
	
	pred_test = lgb_model.predict_proba(test[feature_names], num_iteration=lgb_model.best_iteration_)
	
	prediction[ycol] += pred_test[:, 1] / kfold.n_splits
	
	df_importance = pd.DataFrame({
		'column': feature_names,
		'importance': lgb_model.feature_importances_,
	})
	df_importance_list.append(df_importance)
	
	del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
	gc.collect()


Fold_1 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[67]	train's auc: 0.84272	valid's auc: 0.5254

Fold_2 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[6]	train's auc: 0.726116	valid's auc: 0.535105

Fold_3 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[83]	train's auc: 0.859523	valid's auc: 0.538554

Fold_4 Training ================================

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2]	train's auc: 0.651235	valid's auc: 0.540432

Fold_5 Training ================================

Training until validation scores don't improve for 50 rounds
[500]	train's auc: 0.949987	valid's auc: 0.545005
Early stopping, best iteration is:
[607]	train's auc: 0.956

In [49]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg(
    'mean').sort_values(ascending=False).reset_index()
df_importance

column  importance
0                           op_bias    2770.700
1                          ts_diff1    2632.600
2                             yanzh    1127.300
3                                op     993.100
4                         user_name     635.500
5            ts_diff1_sum_user_name     402.300
6           ts_diff1_mean_user_name     357.100
7            ts_diff1_max_user_name     205.600
8     ts_diff1_mean_browser_version     165.300
9                      browser_type     161.800
10                            loc_3     151.300
11                  browser_version     148.600
12     ts_diff1_max_browser_version     126.400
13                           ip_cat      94.200
14           ts_diff1_std_user_name      74.800
15     ts_diff1_std_browser_version      73.100
16             ts_diff1_mean_ip_cat      72.700
17     ts_diff1_sum_browser_version      61.500
18              ts_diff1_max_ip_cat      59.800
19                            loc_1      45.400
20           ts_diff1_mean_location      38.000
21                            loc_2      37.500
22                          past_23      33.900
23                     device_model      26.200
24            ts_diff1_max_location      24.600
25              ts_diff1_sum_ip_cat      18.000
26                       os_version      17.200
27            ts_diff1_sum_location      15.800
28       ts_diff1_mean_device_model      13.600
29        ts_diff1_median_user_name      13.000
30        ts_diff1_sum_device_model       9.500
31              ts_diff1_std_ip_cat       7.400
32           ts_diff1_median_ip_cat       6.500
33         ts_diff1_mean_os_version       6.400
34         ts_diff1_median_location       5.100
35          ts_diff1_sum_os_version       4.400
36            ts_diff1_std_location       2.900
37                          os_type       1.500
38        ts_diff1_max_device_model       1.100
39     ts_diff1_median_device_model       1.100
40  ts_diff1_median_browser_version       1.100
41          ts_diff1_max_os_version       0.800
42       ts_diff1_median_os_version       0.600
43          ts_diff1_std_os_version       0.100
44        ts_diff1_min_device_model       0.000
45     ts_diff1_min_browser_version       0.000
46              ts_diff1_min_ip_cat       0.000
47            ts_diff1_min_location       0.000
48          ts_diff1_min_os_version       0.000
49    ts_diff1_prod_browser_version       0.000
50       ts_diff1_prod_device_model       0.000
51             ts_diff1_prod_ip_cat       0.000
52           ts_diff1_prod_location       0.000
53         ts_diff1_prod_os_version       0.000
54          ts_diff1_prod_user_name       0.000
55        ts_diff1_std_device_model       0.000
56           ts_diff1_min_user_name       0.000

In [50]:
df_oof = pd.concat(oof)
print('roc_auc_score', roc_auc_score(df_oof[ycol], df_oof['pred']))

roc_auc_score 0.5278499069385952


In [34]:
prediction['id'] = range(len(prediction))
prediction['id'] = prediction['id'] + 1
prediction = prediction[['id', 'risk_label']].copy()
prediction.columns = ['id', 'ret']
prediction.head()

id   ret
6147   1 0.265
6148   2 0.421
6149   3 0.449
6150   4 0.530
6151   5 0.460

In [35]:
prediction.to_csv('G:/kaggle竞赛GM之路！！！/竞赛ing/2021CCF系统认证风险预测/submit_7/submission.csv', index=False)